In [ ]:
pip install pyspark==3.3.1

In [2]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

In [7]:
df = spark.read.json("../../../datalake/twitter_datascience")

In [8]:
df.show()

+--------------------+--------------------+--------------------+------------+
|                data|            includes|                meta|extract_date|
+--------------------+--------------------+--------------------+------------+
|[{737142202481016...|{[{2016-05-30T04:...|{1614960801781252...|  2023-01-16|
|[{156551075009305...|{[{2022-09-02T01:...|{1614960439531819...|  2023-01-16|
|[{161308754706814...|{[{2023-01-11T08:...|{1614960279242276...|  2023-01-16|
|[{109402247573690...|{[{2019-02-08T23:...|{1614960021451976...|  2023-01-16|
|[{101181765595789...|{[{2018-06-27T03:...|{1614959798633795...|  2023-01-16|
|[{137155121135112...|{[{2021-03-15T19:...|{1614959535180898...|  2023-01-16|
|[{142970475005707...|{[{2021-08-23T07:...|{1614959337641762...|  2023-01-16|
|[{18570502, 16149...|{[{2009-01-03T02:...|{1614959152417079...|  2023-01-16|
|[{161308682069857...|{[{2023-01-11T08:...|{1614958997207154...|  2023-01-16|
|[{130630626775391...|{[{2020-09-16T18:...|{1614958862225838...|

In [10]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- impression_count: long (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    

In [11]:
from pyspark.sql import functions as f

In [12]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- impression_count: long (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [13]:
df.select(f.explode("data")).show()

+--------------------+
|                 col|
+--------------------+
|{7371422024810168...|
|{2603796666, 1614...|
|{29726352, 161496...|
|{555031989, 16149...|
|{1608405925886304...|
|{257840966, 16149...|
|{1041973326351151...|
|{1565510750093058...|
|{1565510750093058...|
|{1440322201576636...|
|{1565510750093058...|
|{1565510750093058...|
|{1565510750093058...|
|{1565510750093058...|
|{1613087547068145...|
|{1565510750093058...|
|{3267250213, 1614...|
|{619948731, 16149...|
|{1094022475736903...|
|{1613087547068145...|
+--------------------+
only showing top 20 rows



In [14]:
df.select(f.explode("data").alias("tweets"))\
    .select("tweets.author_id", "tweets.conversation_id",
    "tweets.created_at", "tweets.id",
    "tweets.public_metrics.*", "tweets.text").printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- impression_count: long (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [21]:
tweet_df = df.select(f.explode("data").alias("tweets"))\
    .select("tweets.author_id", "tweets.conversation_id",
    "tweets.created_at", "tweets.id",
    "tweets.public_metrics.*", "tweets.text")

In [22]:
tweet_df.show()

+-------------------+-------------------+--------------------+-------------------+----------------+----------+-----------+-----------+-------------+--------------------+
|          author_id|    conversation_id|          created_at|                 id|impression_count|like_count|quote_count|reply_count|retweet_count|                text|
+-------------------+-------------------+--------------------+-------------------+----------------+----------+-----------+-----------+-------------+--------------------+
| 737142202481016832|1614960801781252098|2023-01-16T12:20:...|1614960801781252098|               0|         0|          0|          0|           56|RT @Khulood_Alman...|
|         2603796666|1614960729500835846|2023-01-16T12:20:...|1614960729500835846|               0|         0|          0|          0|          257|RT @DataScienceDo...|
|           29726352|1614960686169300992|2023-01-16T12:20:...|1614960686169300992|              96|         0|          0|          0|            0|De